In [1]:
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF,IDF
from pyspark.ml.feature import StringIndexer


from pyspark.ml.classification import NaiveBayes

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

In [2]:
os.environ['JAVA_HOME'] = '/opt/jdk' #Mostra aonde está o JDK

In [3]:
spark = SparkSession.builder.master("local[4]") \
                    .appName('ml-teste') \
                    .getOrCreate()

sc=spark.sparkContext

21/10/28 14:18:18 WARN Utils: Your hostname, abner-Lenovo-G40-70 resolves to a loopback address: 127.0.1.1; using 192.168.15.20 instead (on interface wlp2s0)
21/10/28 14:18:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/28 14:18:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
caminho_df_test = 'data_test.csv'
caminho_df_training = 'data_training.csv'

df_test = spark.read.csv(caminho_df_test,header=True)
df_training = spark.read.csv(caminho_df_training, header=True)

In [6]:
label_stringIdx = StringIndexer(inputCol = "topico", outputCol = "label")
tokenization = Tokenizer(inputCol="texto", outputCol="palavras")
remover_stopword = StopWordsRemover(inputCol="palavras", outputCol="palavras_filtradas")
hashingTF = HashingTF(inputCol="palavras_filtradas", outputCol="tf_features")
idf = IDF(inputCol="tf_features", outputCol="tf_idf_features")

# Naive Bays

## nbModel

In [7]:
nb = NaiveBayes(featuresCol='tf_idf_features', labelCol='label', smoothing=1)
pipelineNB = Pipeline(stages=[label_stringIdx, tokenization, remover_stopword, hashingTF, idf, nb])

nbModel = pipelineNB.fit(df_training) #treina o modelo nb_model

21/10/28 14:19:21 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:19:25 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


## nbcvModel

In [11]:
# Create ParamGrid for Cross Validation
nbparamGrid = ParamGridBuilder() \
               .addGrid(nb.smoothing, [0.0, 0.2, 0.4]) \
               .build()

In [12]:
# Create 5-fold CrossValidator
nbcv = CrossValidator(estimator = pipelineNB,
                      estimatorParamMaps = nbparamGrid,
                      evaluator = MulticlassClassificationEvaluator(predictionCol="prediction"),
                      numFolds = 3)

In [13]:
nbcvModel = nbcv.fit(df_training)

21/10/28 14:23:18 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:19 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:26 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB
21/10/28 14:23:30 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:30 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:36 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB
21/10/28 14:23:39 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:40 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:45 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB
21/10/28 14:23:49 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:50 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 14:23:57 WARN DAGScheduler: Broadcasting l

## nbModel com df_test

In [14]:
df_test_nbModel = nbModel.transform(df_test)

In [ ]:
#df_test_nbModel.show()

In [ ]:
#df_test_nbModel.printSchema()

## nbcvModel com df_test

In [16]:
df_test_nbcvModel = nbcvModel.transform(df_test)

## Evaluation

In [17]:
df_test_nbModel = df_test_nbModel.select(['prediction', 'label'])
rdd_pred_label_nbModel = df_test_nbModel.rdd 
metrics_nbModel = MulticlassMetrics(rdd_pred_label_nbModel)

df_test_nbcvModel = df_test_nbcvModel.select(['prediction', 'label'])
rdd_pred_label_nbcvModel = df_test_nbcvModel.rdd 
metrics_nbcvModel = MulticlassMetrics(rdd_pred_label_nbcvModel)

21/10/28 14:36:16 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB
21/10/28 14:36:19 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB


In [23]:
vec_macroF1 = []
qtdClasses = 0

labels = rdd_pred_label_nbModel.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("\n\tClass %d precision = %s" % (label, metrics_nbModel.precision(label)))
    print("\tClass %d recall = %s" % (label, metrics_nbModel.recall(label)))
    print("\tClass %d F1 Measure = %s" % (label, metrics_nbModel.fMeasure(label, beta=1.0)))
    vec_macroF1.append(metrics_nbModel.fMeasure(label, beta=1.0))
    qtdClasses +=1

macroF1 = (sum(vec_macroF1))/qtdClasses
print("\n\tmacroF1 = %s" % macroF1)

print("\n\tAccuracy = %s" % metrics_nbModel.accuracy)

21/10/28 15:06:57 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB



	Class 0 precision = 0.6016949152542372
	Class 0 recall = 1.0
	Class 0 F1 Measure = 0.7513227513227513

	Class 1 precision = 0.7222222222222222
	Class 1 recall = 0.9285714285714286
	Class 1 F1 Measure = 0.8125000000000001

	Class 2 precision = 1.0
	Class 2 recall = 1.0
	Class 2 F1 Measure = 1.0

	Class 3 precision = 1.0
	Class 3 recall = 0.1
	Class 3 F1 Measure = 0.18181818181818182

	Class 4 precision = 0.0
	Class 4 recall = 0.0
	Class 4 F1 Measure = 0.0

	Class 5 precision = 1.0
	Class 5 recall = 0.15384615384615385
	Class 5 F1 Measure = 0.2666666666666667

	Class 6 precision = 1.0
	Class 6 recall = 0.5
	Class 6 F1 Measure = 0.6666666666666666

	Class 7 precision = 0.0
	Class 7 recall = 0.0
	Class 7 F1 Measure = 0.0

	Class 8 precision = 0.0
	Class 8 recall = 0.0
	Class 8 F1 Measure = 0.0

	macroF1 = 0.4087749184971407

	Accuracy = 0.6779661016949152


In [24]:
vec_macroF1 = []
qtdClasses = 0

labels = rdd_pred_label_nbcvModel.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("\n\tClass %d precision = %s" % (label, metrics_nbcvModel.precision(label)))
    print("\tClass %d recall = %s" % (label, metrics_nbcvModel.recall(label)))
    print("\tClass %d F1 Measure = %s" % (label, metrics_nbcvModel.fMeasure(label, beta=1.0)))
    vec_macroF1.append(metrics_nbcvModel.fMeasure(label, beta=1.0))
    qtdClasses +=1

macroF1 = (sum(vec_macroF1))/qtdClasses
print("\n\tmacroF1 = %s" % macroF1)

print("\n\tAccuracy = %s" % metrics_nbModel.accuracy)   

21/10/28 15:07:27 WARN DAGScheduler: Broadcasting large task binary with size 22.1 MiB



	Class 0 precision = 0.696969696969697
	Class 0 recall = 0.971830985915493
	Class 0 F1 Measure = 0.811764705882353

	Class 1 precision = 0.7
	Class 1 recall = 1.0
	Class 1 F1 Measure = 0.8235294117647058

	Class 2 precision = 1.0
	Class 2 recall = 1.0
	Class 2 F1 Measure = 1.0

	Class 3 precision = 1.0
	Class 3 recall = 0.1
	Class 3 F1 Measure = 0.18181818181818182

	Class 4 precision = 0.6
	Class 4 recall = 0.21428571428571427
	Class 4 F1 Measure = 0.3157894736842105

	Class 5 precision = 1.0
	Class 5 recall = 0.6923076923076923
	Class 5 F1 Measure = 0.8181818181818181

	Class 6 precision = 1.0
	Class 6 recall = 0.8
	Class 6 F1 Measure = 0.888888888888889

	Class 7 precision = 0.0
	Class 7 recall = 0.0
	Class 7 F1 Measure = 0.0

	Class 8 precision = 0.0
	Class 8 recall = 0.0
	Class 8 F1 Measure = 0.0

	macroF1 = 0.537774720024462

	Accuracy = 0.6779661016949152


In [20]:
nbcvModel.avgMetrics #Não seio o que é essa métrica, pesquisar depois.

[0.3236290375876557, 0.6513435678152694, 0.6511170994265941]